In [ ]:
import DeepFMKit.core as dfm
import DeepFMKit.plotting as plot
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as sc
from spectools.lpsd import psd

# Instantiate the main framework
dff = dfm.DeepFitFramework()

# --- 1. Define the Single, Shared Laser Source ---
laser_config = dfm.LaserConfig(label="main_laser")
laser_config.f_mod = 1000 # Modulation frequency (Hz)
laser_config.f_n = 1e6 # Laser frequency noise at 1 Hz (Hz/rtHz)
laser_config.amp_n = 1e-5 # Laser amplitude noise (1/rtHz)

# --- 2. Define the Main Interferometer ---
main_ifo_config = dfm.InterferometerConfig(label="dynamic_ifo")
main_ifo_config.ref_arml = 0.1 # Reference arm length (m)
main_ifo_config.meas_arml = 0.3 # Measurement arm length (m)
main_ifo_config.arml_mod_f = 1.0 # Measurement arm modulation frequency (Hz)
main_ifo_config.arml_mod_amp = 1e-8 # Armlength modulation amplitude (m)
main_ifo_config.arml_mod_n = 1e-12 # Armlength modulation amplitude noise (m/rtHz)

# --- 3. Set Modulation Depth by Adjusting Laser's `df` ---
m_target = 6.0 # Target effective modulation index (rad)
opd = main_ifo_config.meas_arml - main_ifo_config.ref_arml # Optical pathlength difference (m)
df_required = (m_target * sc.c) / (2 * np.pi * opd) # Required laser modulation amplitude (Hz)
laser_config.df = df_required

# --- 4. Compose the Main Channel ---
main_label = "dynamic_channel"
main_channel = dfm.DFMIObject(
    label=main_label,
    laser_config=laser_config,
    ifo_config=main_ifo_config,
    f_samp=int(200e3) # Sampling frequency (Hz)
)
dff.sims[main_label] = main_channel

# --- 5. Create a Witness Channel ---
# This single line replaces the manual creation and configuration.
ref_label = "reference_channel"
dff.create_witness_channel(
    main_channel_label=main_label, 
    witness_channel_label=ref_label, 
    m_witness=4.3 # We want a witness with this effective modulation index
)

# --- 6. Simulate ---
dff.simulate(
    main_label=main_label, 
    witness_label=ref_label, 
    n_seconds=100
)

# --- 7. Analyze and Plot ---
print("\n--- Configuration Summaries ---")
dff.sims[main_label].info()
dff.sims[ref_label].info()

print("\n--- Fitting Channels ---")
for i, key in enumerate(dff.raws):
    print(f"Fitting channel '{key}'...")
    dff.fit(key, fit_label=f"fit_{key}")

print("\n--- Plotting Results ---")
ax = dff.plot()
ax[0].legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))

ax.plot(dff.raws["dynamic_channel"].phi_sim_downsamp-np.mean(dff.raws["dynamic_channel"].phi_sim_downsamp))
ax.plot(dff.fits['fit_dynamic_channel'].phi-dff.fits['fit_reference_channel'].phi-np.mean(dff.fits['fit_dynamic_channel'].phi-dff.fits['fit_reference_channel'].phi))

plt.show()

In [ ]:
f_fit = dff.sims["reference_channel"].f_fit

f2, psd2 = psd(dff.fits['fit_dynamic_channel'].phi, fs=f_fit, bmin=1)
f3, psd3 = psd(dff.fits['fit_reference_channel'].phi, fs=f_fit, bmin=1)
f4, psd4 = psd(dff.fits['fit_dynamic_channel'].phi - dff.fits['fit_reference_channel'].phi, fs=f_fit, bmin=1)
f5, psd5 = psd((dff.fits['fit_dynamic_channel'].phi - dff.fits['fit_reference_channel'].phi) + dff.raws["dynamic_channel"].phi_sim_downsamp, fs=f_fit, bmin=1)

ax = plot.asd_plot([f2, f3, f4, f5], [psd2, psd3, psd4, psd5], label_list=["ch0", "ch1 (ref)", "ch0 - ch1", "ch0 - ch1 - phi"], psd=True)
plt.show()